### Housekeeping 

For Fall 2019, 2020, 2021, and 2022, we had student data from the IR database for Day 1, 20th-Day, and EOT. I downloaded all three recordings of students and then eliminated the duplicates by student ID. The reason I chose to do this rather than the more traditional way in the research, which is usually Day 1 to Day 1 or 20th-Day to 20th-Day is because I want as complete a picture as possible of the students who persist from one Fall to the next. Most research focuses on First Time-Full Time, which is the IPEDs data. On a practical level, we are interested in what predictors help us predict all of the students who reenroll from one semester to the next. For Fall 2023, since we do not have all three sets, I downloaded the data from Argos as well as the Day 1 data from the IR database and eliminated duplicate IDs. By the time I finish the report, we will have 20th-Day numbers. I will incorporate those in and that will be the best I can do for this semester.

The following attributes were used in the analysis:

* Earned_sem_crhr
* Tot_inst_hrs
* term_gpa
* inst_gpa
* pell/no_pell
* scholarship
* ethnicity
* age
* distance (long, lat: Haversine Formula)
* Stype
* attempted hours
* interaction term (earned/att = completion ratio)
* fully online vs. not fully online

In [ ]:
import pandas as pd
import numpy as np

import logging
import sys
from pathlib import Path

import ipytest

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(exist_ok=True)
sys.path.extend([f"./{CODE_FOLDER}"])

file_path = ['Files/', ' Enrollment.csv']
file_path_gpa = ['Files/', ' GPA and CrHrs.csv']

semesters = ['201980', '202080', '202180', '202280', '202380']

In [ ]:
%%html
<style>
table {float:left}
</style>

In [ ]:
# Load in all semesters' enrollment

all_sems = []
for i in semesters:
    temp = pd.read_csv(file_path[0] + i + file_path[1])
    temp.columns = [i.lower() for i in temp.columns]
    all_sems.append(temp)
    
all_sems = (pd.concat(all_sems)
              .reset_index(drop = True)
           )

In [ ]:
# load in the gpa and attempted/earned credit hour datasets
all_gpas = []

for i in semesters[:4]:
    temp = (pd.read_csv(file_path_gpa[0] + i + file_path_gpa[1])
              .rename(columns = str.lower))
    all_gpas.append(temp)

all_gpas = (pd.concat(all_gpas).reset_index(drop = True)
              .rename(columns = {'studentid':'id',
                                 'gpatrm':'term'})
           )

# There was one single row out of 30000 that had NaN values so 
# I filtered it out.
all_gpas = all_gpas[all_gpas['term'].isna() == False]

all_gpas['term'] = all_gpas['term'].astype(int)

In [ ]:
# Load Pell Grant and Load information
pell = (pd.read_csv(file_path[0] + 'Pell and Loan FA19 - FA23.csv')
          .rename(columns = {'LOAN_GRANT_TERM':'term'})
          .rename(columns = str.lower)
       )

# Sort dataframe
pell = pell.sort_values(['term', 'id'])

In [ ]:
from processing import select_sem, find_enrolled

# Make list of years
sems_filt = [19, 20, 21, 22, 23]

# Loop through each semester to compare them to one another and 
# record who enrolled from one semester to the next.
perc_enrolled = []
all_enrolled = []

for i in range(1, 5, 1):
    temp_perc = find_enrolled(select_sem(all_sems, int(semesters[i-1])), 
                         select_sem(all_sems, int(semesters[i])), 
                         sems_filt[i - 1], sems_filt[i])[0]
    temp_enrolled = find_enrolled(select_sem(all_sems, int(semesters[i-1])), 
                         select_sem(all_sems, int(semesters[i])), 
                         sems_filt[i - 1], sems_filt[i])[1]
    perc_enrolled.append(temp_perc)
    all_enrolled.append(temp_enrolled)

In [ ]:
(pd.concat(perc_enrolled)
   .pivot_table(index = 'terms', columns = 'enrolled', values = ['cnt', 'percent'])
)

In [ ]:
# Combine the all semesters of with the enrolled/unenrolled students in it
all_enrolled_df = (pd.concat(all_enrolled)
                     .reset_index(drop = True)
                  )

In [ ]:
# Combine the dataframe with the student data and enrollment status along
# with the gpas and credit hour data
enrolled_gpas = all_enrolled_df.merge(all_gpas, how = 'left', on = ['id', 'term'])

# There are some missing values from students who did not make it to the EOT and therefore
# do not have GPAs. The code below is to see who those students are and to determine if 
# eliminiating them is going to severly skew the data.
temporary_df = (pd.DataFrame(enrolled_gpas[enrolled_gpas['overall_gpa'].isna()].groupby(['term', 'enrolled'])['id'].count())
                 .reset_index()
                 .rename(columns = {'id':'cnt'})
               )

ls = []
for i in temporary_df['term'].unique():
    temp = temporary_df[temporary_df['term'] == i]
    ls.append(temp['cnt']/temp['cnt'].sum())
    

temporary_df['percent'] = list(pd.concat(ls))

# now, the dataframe after removing the missing values
# This allows me to compare the two and see how the distribution
# changed. If it changed significantly, I will need to examine them
# more fully.
more_temporary = (pd.DataFrame(enrolled_gpas[enrolled_gpas['overall_gpa'].isna()==False].groupby(['term', 'enrolled'])['id'].count())
                   .reset_index()
                   .rename(columns = {'id':'cnt'})
                 )

more_temporary_ls = []
for i in temporary_df['term'].unique():
    temp = more_temporary[more_temporary['term'] == i]
    more_temporary_ls.append(temp['cnt']/temp['cnt'].sum())
    
more_temporary['percent'] = list(pd.concat(more_temporary_ls))

more_temporary.pivot_table(index = 'term', columns = 'enrolled', values = ['cnt', 'percent'])


### Comparison of the distribution of enrolled vs not enrolled

This compares the distributions before I added the gpa dataset and after. After I merged the two, there were missing values. This shows how it alters the distribution if I just remove the missing values. It slightly elevates the enrolled while slightly lowering the unenrolled.

**Table 1**
|**terms**|**Enrolled**|**Not Enrolled**|**Enrolled**|**Not Enrolled**|
|:--------|-----------:|---------------:|-----------:|---------------:|
|fa19_fa20| 	3296| 	5139| 	0.390753| 	0.609247|
|fa20_fa21| 	3031| 	4628| 	0.395744| 	0.604256|
|fa21_fa22| 	2984| 	4502| 	0.398611| 	0.601389|
|fa22_fa23| 	2770| 	4354| 	0.388827| 	0.611173|

<br><br><br><br><br><br><br>

**Table 2**
        
|**term**|**Enrolled**|**Not Enrolled**|**Enrolled**|**Not Enrolled**|
|:-------|-----------:|---------------:|-----------:|---------------:|
|201980| 	3245| 	4781| 	0.404311| 	0.595689|
|202080| 	2965| 	4329| 	0.406498| 	0.593502|
|202180| 	2931| 	4165| 	0.413050| 	0.586950|
|202280| 	2710| 	4058| 	0.400414| 	0.599586|

<br><br><br><br><br><br><br>

**Table 3**

|**term**|**Enrolled**|**Not Enrolled**|**Enrolled**|**Not Enrolled**|
|:-------|-----------:|---------------:|-----------:|---------------:|
|201980|	3229| 	4749|	0.404738| 	0.595262|
|202080| 	2897| 	4256| 	0.405005| 	0.594995|
|202180| 	2912| 	4137| 	0.413108| 	0.586892|
|202280| 	2697| 	4040| 	0.400327| 	0.599673|

### Creating Final Dataset For Evaluation

In [ ]:
# Reorganize the columns to 
enrolled_gpas = enrolled_gpas[['term', 'pidm', 'age', 'id', 'totcr', 'status', 'stype', 'resd_desc',
                               'degcode', 'majr_desc1', 'gender', 'mrtl', 'ethn_desc', 'cnty_desc1',
                               'styp', 'resd', 'acd_std_desc', 'term_att_crhr', 'term_earn_crhr', 
                               'term_gpa', 'inst_gpa', 'inst_earned', 'inst_hrs_att', 'overall_gpa',
                               'enrolled']]

# Remove NaN values
enrolled_gpas = enrolled_gpas[enrolled_gpas['overall_gpa'].isna() == False].reset_index(drop = True)

### Missing Data

In [ ]:
# Look over missing values. I made the foolish mistake when I was first downloading the data
# to label missing values as'Not Enrolled'.
d = {}
for i in enrolled_gpas.columns:
    temp = enrolled_gpas[enrolled_gpas[i] == 'Not Enrolled']
    d[i] = (len(temp)/len(enrolled_gpas)) * 100

(pd.DataFrame.from_dict(d, orient = 'index')
   .reset_index().rename(columns = {'index':'col_names',
                                    0:'percent_missing'})
)

output = None

### Evaluating Missing Data.

As has been the case with other student data I have evalulated through the years for Butler, the *marital* data continues to be woefully lacking, to the extent that it is unusable in any real sense. That attribute will be removed. The *gender* column is missing $\frac{1}{100}^{th}$ of a percent. I will just delete those values. *County* data is missing $\frac{9}{10}^{ths}$ of one percent, so that will be removed as well. Finally, *ethnicity* is missing 6.4453%, which is congruent with previous analysis. I will impute a new attribute there simply labeled, "missing."

|**col_names**|**percent_missing**|
|:------------|------------------:|
|term| 	0.000000|
|pidm| 	0.000000|
|age |	0.000000|
|id |   0.000000|
|totcr 	|0.000000|
|status |	0.000000|
|stype 	|0.000000|
|resd_desc| 	0.000000|
|degcode |	0.000000|
|majr_desc1| 	0.000000|
|gender |	0.010280|
|mrtl 	|51.117050|
|ethn_desc| 	6.445312|
|cnty_desc1| 	0.904605|
|styp 	|0.000000|
|resd 	|0.000000|
|acd_std_desc| 	0.000000|
|term_att_crhr| 	0.000000|
|term_earn_crhr| 	0.000000|
|term_gpa 	|0.000000|
|inst_gpa 	|0.000000|
|inst_earned |	0.000000|
|inst_hrs_att |	0.000000|
|overall_gpa |	0.000000|
|enrolled |	59.392133|

In [ ]:
# Remove the 'mrtl' column
enrolled_gpas = enrolled_gpas[['term', 'pidm', 'age', 'id', 'totcr', 'status', 'stype', 'resd_desc',
                               'degcode', 'majr_desc1', 'gender', 'ethn_desc', 'cnty_desc1', 'styp', 
                               'resd', 'acd_std_desc', 'term_att_crhr', 'term_earn_crhr', 'term_gpa', 
                               'inst_gpa', 'inst_earned', 'inst_hrs_att', 'overall_gpa', 'enrolled']]

# Remove missing 'gender' and 'cnty_desc1' values
enrolled_gpas = enrolled_gpas[enrolled_gpas['gender'] != 'Not Enrolled']
enrolled_gpas = enrolled_gpas[enrolled_gpas['cnty_desc1'] != 'Not Enrolled']

# Impute missing values in 'ethn_desc' with the attribute 'Missing'
enrolled_gpas['ethn_desc'] = enrolled_gpas['ethn_desc'].replace('Not Enrolled', 'Missing')

# Take another look at the data to make sure there are no more missing values
d = {}
for i in enrolled_gpas.columns:
    temp = enrolled_gpas[enrolled_gpas[i] == 'Not Enrolled']
    d[i] = (len(temp)/len(enrolled_gpas)) * 100

(pd.DataFrame.from_dict(d, orient = 'index')
   .reset_index().rename(columns = {'index':'col_names',
                                    0:'percent_missing'})
)

output = None

In [ ]:
# View the final distribution of enrolled/not enrolled to see how it has been altered by
# eliminating the NaNs. This was put up with the other two tables as "Table 3"
more_temporary = pd.DataFrame(enrolled_gpas.groupby(['term', 'enrolled'])['id'].count())\
                   .reset_index()\
                   .rename(columns = {'id':'cnt'})

more_temporary_ls = []
for i in temporary_df['term'].unique():
    temp = more_temporary[more_temporary['term'] == i]
    more_temporary_ls.append(temp['cnt']/temp['cnt'].sum())
    
more_temporary['percent'] = list(pd.concat(more_temporary_ls))

more_temporary.pivot_table(index = 'term', columns = 'enrolled', values = ['cnt', 'percent'])

Output = None

In [ ]:
%store enrolled_gpas

# Online Classes Online
The section below is to identify students who took all online classes as designated by their location data rather than by what they claimed on their enrollment form with the registrar. 

In [ ]:
# Import FA19 - FA22
fa19_fa22 = (pd.read_csv('FA19 - FA22 CrHr Enrollment.csv')
               .rename(columns = str.lower)
            )


In [ ]:
from processing import count_online_classes

# Loop through all the previous Fall semesters and
# use the count_online_classes() function
all_sems_online = []

for i in [201980, 202080, 202180, 202280]:
    temp = count_online_classes(fa19_fa22, i)
    all_sems_online.append(temp)

In [ ]:
# Make dataframe of the terms with their counts of fully online and not fully online
fully_online_cnts = (pd.DataFrame(pd.concat(all_sems_online).reset_index(drop = True).groupby(['term', 'fully_online'])['id'].count())
                       .reset_index()
                       .rename(columns = {'id':'count'})
                    )
fully_online_cnts

I went through several of these through SFASRPO to make sure the count of classes was correct as well as the number of clases that were online and the percentage that represented of the ovarll total. It all checks out. So now it needs to be incorporated back into the main dataframe.

In [ ]:
pd.concat(all_sems_online)[:50]

output = None

In [ ]:
# Pull together all semesters' fully online data
fully_online = pd.concat(all_sems_online).reset_index(drop = True)

# Isolatejust the columns I need
fully_online = fully_online[['id', 'term', 'fully_online']]

# Merge enrolled_gpas and fully_online datasets
enrolled_gpas_online = enrolled_gpas.merge(fully_online, how = 'left', on = ['id', 'term'])\
                       [['term', 'pidm', 'age', 'id', 'totcr', 'status', 'stype', 'resd_desc',
                        'degcode', 'majr_desc1', 'gender', 'ethn_desc', 'cnty_desc1', 'styp',
                        'resd', 'acd_std_desc', 'term_att_crhr', 'term_earn_crhr', 'term_gpa',
                        'inst_gpa', 'inst_earned', 'inst_hrs_att', 'overall_gpa', 'fully_online',
                        'enrolled']]

In [ ]:
enrolled_gpas_online

output = None

# Pell Grant and Loans

One of the things that was unexpected was how many different offerings students have. The maximum number of FAFSA offerings between scholarships, loands, and grants for a single student was 12. There was one student for all five Fall semesters I am looking at that had 12 FAFSA offerings. The average number of FAFSA offers is 3.

In [ ]:
# load data
pell.head()

In [ ]:
pell['term'].unique()

In [ ]:
# Count of FAFSA offerings, a count of scholarships, pell, and loans
# offered to each student

d = {}

for i in pell['term'].unique():
    temp = pell[pell['term'] == i]
    for j in temp['id'].unique():
        temp2 = temp[temp['id'] == j]
        d[str(i) + j] = len(temp2)

In [ ]:
fafsa_count = (pd.DataFrame.from_dict(d, orient = 'index')
                 .reset_index()
                 .rename(columns = {'index':'term_id',
                                    0:'count_of_fafsa'})
              )
    
(fafsa_count[fafsa_count['count_of_fafsa'] >= fafsa_count['count_of_fafsa'].max() - 1]
    .reset_index(drop = True)
    .sort_values('count_of_fafsa', ascending = False)
)

## Accepted FAFSA Monies

Crucially, I didn't want to count FAFSA offered that does not get accepted. I determined this by filtering the 'accepted amount' by all the values that are not zero and filtering only thosse that had a 'paid date' that was not *NaN*. This leaves me with all the values that are non-zero and were accepted. When I did this, the top number of Fafsa awards was 10 in 202380 and 9 for 201980, 202080, 202180, and 202280. By using this selection process, the number of possible FAFSA awards over the five years dropped from 91436 to 41312, a difference of 50124.

I also created a new column name called *all_fafsa* that is a sum across the row of each *id* for each *term* of awarded financial aid. This variable counts the total number of pell grants, loans, and scholarships a student has for each semester.

In [ ]:
pell['accept_amt'] = pell['accept_amt'].fillna(0).astype(int)

# Filter only those that had a pay out date
pell_mask1 = pell['paid_date'].isna() == False

# Filter only those that had a monetary payout
pell_mask2 = pell['accept_amt'] != 0.00

# Use filters to create the accepted FAFSA
pell_accepted = pell[pell_mask1 & pell_mask2].reset_index(drop = True)

print('The length of the "pell" dataframe verses the "pell_accepted" dataframe is:\n',
      "'pell': ", len(pell), '\n',
      "'pell_accepted': ", len(pell_accepted), '\n',
      " Difference: ", len(pell) - len(pell_accepted), sep = "")

pell_accepted.head(10)

In [ ]:
# Create dummy variables for each level of the pell_nopell column

dummies = []

for i in pell_accepted['term'].unique():
    # Create temporary DF for each term
    temp = pell_accepted[pell_accepted['term'] == i]
    
    # Convert each semester's 'pell_nopell' column to indicator variables
    temp_dummies = pd.get_dummies(temp['pell_nopell'])
    
    # Combine the id and term with the dummy variables
    temp_final = pd.concat([temp[['id', 'term']], temp_dummies], axis = 1)
    
    # Groupby 'id' and 'term' and sum
    grouped_by_id_term = temp_final.groupby(['id', 'term']).sum().reset_index()
    
    # Do the summing process and last time with the groupby function
    temp_final = grouped_by_id_term.groupby('id').sum().reset_index()
    
    # Save the temp_final to dummies list
    dummies.append(temp_final)
    
# Concatenate dummies list
final_pell = pd.concat(dummies).fillna(0)

In [ ]:
# Convert Summer Plus and Kansas Promise to integers
final_pell['Summer Plus'], final_pell['Kansas Promise'] = final_pell['Summer Plus'].astype(int), final_pell['Kansas Promise'].astype(int)

# Create a column that totals all the aid offered (note this is not accepted and received FinAid, just offered FA)
cols_to_sum = ['NO PELL', 'PELL', 'Subsidized', 'Unsubsidized', 'Summer Plus', 'Kansas Promise']

# Sum all accepted FAFSA money for each ID each semester
final_pell['all_fafsa'] = final_pell[cols_to_sum].sum(axis = 1)


In [ ]:
# Convert all the 'NO PELL' values in the column to either 1 or 0
final_pell['NO PELL'] = [1 if i >= 1 else 0 for i in final_pell['NO PELL']]

# View final pell
final_pell = final_pell.reset_index(drop = True)

final_pell.groupby('term')['id'].count(), enrolled_gpas_online.groupby('term')['id'].count()

### Combine the enrolled_gpas_online dataframe with the final_pell dataframe

In [ ]:
# Merge fafsa dataframe with enrolled_gpas_online dataframe
enrolled_gpas_online_fafsa = (enrolled_gpas_online.merge(final_pell, how = 'left', on = ['id', 'term'])
                                  .rename(columns = {'Summer Plus':'summer_plus',
                                                     'Kansas Promise':'kansas_promise',
                                                     'NO PELL':'no_pell'})
                             )

# Loop thrugh the FAFSA columns and fill all NaN values with 0 and make 
# column into integer
fafsa_cols = list(enrolled_gpas_online_fafsa.columns[25:])

for i in fafsa_cols:
    enrolled_gpas_online_fafsa[i] = enrolled_gpas_online_fafsa[i].fillna(0).astype(int)

In [ ]:
enrolled_gpas_online_fafsa = enrolled_gpas_online_fafsa[['term', 'pidm', 'age', 'id', 'totcr', 'status', 'stype', 'resd_desc',
                               'degcode', 'majr_desc1', 'gender', 'ethn_desc', 'cnty_desc1', 'styp',
                               'resd', 'acd_std_desc', 'term_att_crhr', 'term_earn_crhr', 'term_gpa',
                               'inst_gpa', 'inst_earned', 'inst_hrs_att', 'overall_gpa',
                               'fully_online', 'no_pell', 'PELL', 'Subsidized',
                               'Unsubsidized', 'summer_plus', 'kansas_promise', 'all_fafsa', 'enrolled']]

enrolled_gpas_online_fafsa.columns = [i.lower() for i in enrolled_gpas_online_fafsa.columns]


In [ ]:
len(enrolled_gpas_online_fafsa)

### Add In Matriculated High School Students

In [ ]:
hs_file_path = 'C:/pathway to high school data for Dean Streeter/Files/'

# Load Data
semesters = [201980, 202080, 202180, 202280, 202380]

all_sems = []
for sem in semesters:
    temp = pd.read_csv(hs_file_path + str(sem) + ' Demographic Data.csv')
    all_sems.append(temp)
    
# Combine semesters
all_hs = pd.concat(all_sems).reset_index(drop = True)\
                   .rename(columns = {'STDTID':'ID',
                                      'TERMENTERED':'TERM'})

# Make all columns lowercase
all_hs.columns = [i.lower() for i in all_hs.columns]

# Make hsgraddte into datetime object
all_hs['hsgraddte'] = pd.to_datetime(all_hs['hsgraddte'])

# Make year column for hs grad date
all_hs['hs_grad_yr'] = all_hs['hsgraddte'].dt.year.fillna(0).astype(int)

# Convert term to string to extract term_year
all_hs['term'] = all_hs['term'].astype(str)

all_hs['term_year'] = [all_hs['term'][i][:4] for i in range(len(all_hs))]

all_hs['term_year'] = all_hs['term_year'].astype(int)

# Identify which students enrolled in the Fall right after HS Graduation
all_hs['hs_matriculation'] = ['From HS' if  i == j else 'Not From HS' for i, j in zip(all_hs['term_year'], all_hs['hs_grad_yr'])]

hs_matriculation = (pd.DataFrame(all_hs.groupby(['term', 'hs_matriculation'])['id'].count())
                      .reset_index(drop = False)
                      .rename(columns = {'id':'cnt'})
                   )

hs_matriculation

In [ ]:
# Merge enrolled_gpas_online_fafsa with all_hs[['id', 'term', 'hs_matriculation']]
all_hs_for_merge = all_hs[['id', 'term', 'hs_matriculation']]
all_hs_for_merge['term'] = all_hs_for_merge['term'].astype(int)

enrolled_gpas_online_fafsa_hs = (enrolled_gpas_online_fafsa.merge(all_hs_for_merge, how = 'left', on = ['id', 'term'])
                                    .drop_duplicates(subset = ['id', 'term'])
                                )


In [ ]:
# Check to make sure I didn't lose any students unnecessarily in the merge. 
# For some reason, the merge created a few dozen additional lines, which is why
# I had to add 'drop_duplicates' to it. Therefore, I am making sure I am left 
# with the original set of students I wanted to have.
df1 = enrolled_gpas_online_fafsa[['term', 'id']]
df2 = enrolled_gpas_online_fafsa_hs[['term', 'id']]

# Sort both DataFrames by their columns and reset their indices
df1_sorted = df1.sort_values(by=['term', 'id']).reset_index(drop=True)
df2_sorted = df2.sort_values(by=['term', 'id']).reset_index(drop=True)

if df1_sorted.equals(df2_sorted):
    print("The DataFrames are the same.")
else:
    print("The DataFrames are different.")

In [ ]:
# Reorient columns
enrolled_gpas_online_fafsa_hs = enrolled_gpas_online_fafsa_hs[['term', 'pidm', 'age', 'id', 'totcr', 'status', 'stype', 'resd_desc',
                                   'degcode', 'majr_desc1', 'gender', 'ethn_desc', 'cnty_desc1', 'styp',
                                   'resd', 'acd_std_desc', 'term_att_crhr', 'term_earn_crhr', 'term_gpa',
                                   'inst_gpa', 'inst_earned', 'inst_hrs_att', 'overall_gpa',
                                   'fully_online', 'no_pell', 'pell', 'subsidized', 'unsubsidized',
                                   'summer_plus', 'kansas_promise', 'all_fafsa', 'hs_matriculation', 'enrolled']]
                                   

In [ ]:
enrolled_gpas_online_fafsa_hs.to_csv('FA19 - FA23 Cleaned Dataset.csv', index = False)

In [ ]:
%store enrolled_gpas_online_fafsa_hs